In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import math

In [ ]:

# Carrega modelo BERT pré-treinado e seu tokenizer
modelName = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(modelName)
model = BertModel.from_pretrained(modelName, output_hidden_states=True)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
# Lê o arquivo com frases
with open('frases2.txt','r', encoding='utf-8') as f:
    frases = f.readlines()

In [ ]:
# Aplica o tokenizer nas frases (com padding e truncamento)
tokens = tokenizer(frases, return_tensors='pt', padding=True, truncation=True, max_length=512)

In [ ]:
# Gera os embeddings com o modelo BERT
with torch.no_grad():
    outputs = model(**tokens)

In [ ]:
# Lista de tensores com os embeddings de cada camada (13: emb + 12 camadas)
# Cada tensor tem shape [nFrases, nEmbeddings, 768]
embeddingsPorCamada = outputs.hidden_states

In [ ]:
# Número de camadas, frases e embeddings por frase
nCamadas = len(embeddingsPorCamada)           # 13 camadas
nFrases = embeddingsPorCamada[0].shape[0]     # número de frases
nEmbeddings = embeddingsPorCamada[0].shape[1] # número de embeddings por frase (tokens)

In [ ]:
# Função que calcula o ângulo (graus) entre dois vetores (sem normalizar)
def anguloEntre(v1, v2):
    produtoEscalar = (v1 * v2).sum(dim=-1)
    normaV1 = torch.norm(v1, dim=-1)
    normaV2 = torch.norm(v2, dim=-1)
    cosTheta = (produtoEscalar / (normaV1 * normaV2)).clamp(-1, 1)
    return torch.acos(cosTheta) * (180 / math.pi)


In [ ]:
# Lista para armazenar a curvatura média de cada frase
desvioCurvaturaPorFrase = []


In [ ]:
# Percorre todas as frases
for iFrase in range(nFrases):
    angulosPorEmbedding = []  # lista de listas de ângulos para cada embedding dessa frase

    # Percorre cada embedding (token) da frase
    for iEmbedding in range(nEmbeddings):
        angulos = []

        # Percorre pares de camadas consecutivas
        for iCamada in range(nCamadas - 1):
            vetor1 = embeddingsPorCamada[iCamada][iFrase][iEmbedding]    # vetor da camada i
            vetor2 = embeddingsPorCamada[iCamada + 1][iFrase][iEmbedding] # vetor da camada i+1

            angulo = anguloEntre(vetor1, vetor2)  # calcula o ângulo entre os vetores
            angulos.append(angulo.item())

        angulosPorEmbedding.append(angulos)  # adiciona lista de ângulos desse embedding

    # Calcula a média de curvaturas para a frase
    todosAngulos = [a for angles in angulosPorEmbedding for a in angles]
    mediaFrase = sum(todosAngulos) / len(todosAngulos) if todosAngulos else 0
    desvioCurvaturaPorFrase.append(mediaFrase)

In [ ]:
# Exibe resultados
for i, curvatura in enumerate(desvioCurvaturaPorFrase):
    print(f"Frase {i}: curvatura média dos embeddings = {curvatura:.2f}°")

Frase 0: curvatura média dos embeddings = 27.56°
Frase 1: curvatura média dos embeddings = 27.21°
Frase 2: curvatura média dos embeddings = 27.56°
Frase 3: curvatura média dos embeddings = 27.91°
Frase 4: curvatura média dos embeddings = 27.73°
Frase 5: curvatura média dos embeddings = 27.82°
Frase 6: curvatura média dos embeddings = 28.23°
Frase 7: curvatura média dos embeddings = 27.83°
Frase 8: curvatura média dos embeddings = 28.23°
Frase 9: curvatura média dos embeddings = 27.95°
Frase 10: curvatura média dos embeddings = 27.21°
Frase 11: curvatura média dos embeddings = 27.82°
Frase 12: curvatura média dos embeddings = 28.21°
Frase 13: curvatura média dos embeddings = 28.01°
Frase 14: curvatura média dos embeddings = 28.27°
Frase 15: curvatura média dos embeddings = 27.95°
Frase 16: curvatura média dos embeddings = 27.83°
Frase 17: curvatura média dos embeddings = 28.29°
Frase 18: curvatura média dos embeddings = 28.34°
Frase 19: curvatura média dos embeddings = 28.29°
Frase 20: 

In [ ]:
# Lista para guardar curvaturas espaciais por camada e por frase
curvaturaEspacialPorCamada = []

# Percorre cada camada
for iCamada in range(nCamadas):
    curvaturasFraseAtual = []

    # Percorre cada frase
    for iFrase in range(nFrases):
        angulosEntreEmbeddings = []

        # Percorre pares de embeddings consecutivos da frase
        for iEmbedding in range(nEmbeddings - 1):
            v1 = embeddingsPorCamada[iCamada][iFrase][iEmbedding]     # vetor do token i
            v2 = embeddingsPorCamada[iCamada][iFrase][iEmbedding + 1] # vetor do token i+1

            angulo = anguloEntre(v1, v2)  # ângulo entre os embeddings consecutivos
            angulosEntreEmbeddings.append(angulo.item())

        # Calcula a média dos ângulos da frase para essa camada
        mediaFrase = sum(angulosEntreEmbeddings) / len(angulosEntreEmbeddings) if angulosEntreEmbeddings else 0
        curvaturasFraseAtual.append(mediaFrase)

    # Guarda a lista de médias dessa camada (uma média por frase)
    curvaturaEspacialPorCamada.append(curvaturasFraseAtual)


In [ ]:
for iCamada, mediasFrases in enumerate(curvaturaEspacialPorCamada):
    print(f"Camada {iCamada}:")
    for iFrase, media in enumerate(mediasFrases):
        print(f"  Frase {iFrase}: curvatura espacial média = {media:.2f}°")


Camada 0:
  Frase 0: curvatura espacial média = 66.49°
  Frase 1: curvatura espacial média = 80.99°
  Frase 2: curvatura espacial média = 66.49°
  Frase 3: curvatura espacial média = 72.53°
  Frase 4: curvatura espacial média = 63.79°
  Frase 5: curvatura espacial média = 77.77°
  Frase 6: curvatura espacial média = 52.19°
  Frase 7: curvatura espacial média = 66.91°
  Frase 8: curvatura espacial média = 52.19°
  Frase 9: curvatura espacial média = 72.39°
  Frase 10: curvatura espacial média = 80.99°
  Frase 11: curvatura espacial média = 77.77°
  Frase 12: curvatura espacial média = 58.26°
  Frase 13: curvatura espacial média = 61.64°
  Frase 14: curvatura espacial média = 58.15°
  Frase 15: curvatura espacial média = 72.39°
  Frase 16: curvatura espacial média = 66.91°
  Frase 17: curvatura espacial média = 61.04°
  Frase 18: curvatura espacial média = 52.72°
  Frase 19: curvatura espacial média = 61.04°
  Frase 20: curvatura espacial média = 66.49°
  Frase 21: curvatura espacial méd

In [ ]:
# Lista geral para guardar os ângulos individuais
# Estrutura: [camada][frase][parDeEmbeddings]
angulosIndividuaisPorCamada = []

# Percorre cada camada
for iCamada in range(nCamadas):
    angulosPorFrase = []

    # Percorre cada frase
    for iFrase in range(nFrases):
        angulosEmbeddings = []

        # Percorre pares de embeddings consecutivos
        for iEmbedding in range(nEmbeddings - 1):
            v1 = embeddingsPorCamada[iCamada][iFrase][iEmbedding]
            v2 = embeddingsPorCamada[iCamada][iFrase][iEmbedding + 1]

            angulo = anguloEntre(v1, v2)  # ângulo entre dois embeddings consecutivos
            angulosEmbeddings.append(angulo.item())

        angulosPorFrase.append(angulosEmbeddings)  # salva todos os ângulos dessa frase

    angulosIndividuaisPorCamada.append(angulosPorFrase)  # salva os ângulos da camada


In [ ]:
for iCamada, frases in enumerate(angulosIndividuaisPorCamada):
    print(f"\nCamada {iCamada}:")
    for iFrase, angulos in enumerate(frases):
        print(f"  Frase {iFrase}:")
        for i, angulo in enumerate(angulos):
            print(f"    Embedding {i} → {i+1}: {angulo:.2f}°")



Camada 0:
  Frase 0:
    Embedding 0 → 1: 90.98°
    Embedding 1 → 2: 86.95°
    Embedding 2 → 3: 82.10°
    Embedding 3 → 4: 83.60°
    Embedding 4 → 5: 87.52°
    Embedding 5 → 6: 80.24°
    Embedding 6 → 7: 86.16°
    Embedding 7 → 8: 84.99°
    Embedding 8 → 9: 82.28°
    Embedding 9 → 10: 80.43°
    Embedding 10 → 11: 81.96°
    Embedding 11 → 12: 84.11°
    Embedding 12 → 13: 88.62°
    Embedding 13 → 14: 81.99°
    Embedding 14 → 15: 85.39°
    Embedding 15 → 16: 68.95°
    Embedding 16 → 17: 81.88°
    Embedding 17 → 18: 18.96°
    Embedding 18 → 19: 18.83°
    Embedding 19 → 20: 18.41°
    Embedding 20 → 21: 18.65°
    Embedding 21 → 22: 17.87°
    Embedding 22 → 23: 18.44°
  Frase 1:
    Embedding 0 → 1: 89.01°
    Embedding 1 → 2: 81.42°
    Embedding 2 → 3: 88.33°
    Embedding 3 → 4: 87.03°
    Embedding 4 → 5: 84.60°
    Embedding 5 → 6: 84.70°
    Embedding 6 → 7: 85.60°
    Embedding 7 → 8: 84.67°
    Embedding 8 → 9: 89.13°
    Embedding 9 → 10: 85.93°
    Embedding 1